In [1]:
%run preprocess_kiva.ipynb
"""
以後要改讀取的測資，只需要改這幾個數字
"""
size = (4,4)
n_sp = 7
n_tars = [4]
ids = [2]  
n_robs = [6]
all_data = preprocess_all(ids, size, n_sp, n_tars,n_robs)


for data in all_data:
    print(f"Time Limit (T): {data['T']}")
    print("Start Map (map_start):")
    for row in data['map_start']:
        print(row)
    print("End Map (map_end):")
    for row in data['map_end']:
        print(row)
    print(f"Grid Size: {data['grid_size']}")
    print(f"Number of Cargos: {data['num_cargos']}")
    print(f"Number of Robots: {data['num_robots']}")
    print("Initial Robot Positions (initial_positions):")
    for pos in data['initial_positions']:
        print(pos)
    print("Cargo Initial Positions (cargo_init_pos):")
    for cargo in data['cargo_init_pos']:
        print(cargo)
    print("Cargo Goal Positions (cargo_goal_pos):")
    for goal in data['cargo_goal_pos']:
        print(goal)
    print(f"Alpha: {data['alpha']}")
    print(f"ID: {data['id']}")
    print("\n" + "-"*50 + "\n") 

Time Limit (T): 12
Start Map (map_start):
[0, 0, -1, -1]
[-1, 0, -1, 0]
[1, 3, 0, 4]
[-1, 0, 0, 2]
End Map (map_end):
[1, 0, -1, -1]
[-1, 0, -1, 4]
[0, 0, 0, 2]
[-1, 0, 0, 3]
Grid Size: 4
Number of Cargos: 9
Number of Robots: 6
Initial Robot Positions (initial_positions):
(0, 3, 0)
(1, 1, 0)
(2, 3, 3)
(3, 1, 2)
(4, 1, 3)
(5, 3, 2)
Cargo Initial Positions (cargo_init_pos):
(0, 2, 0)
(1, 3, 3)
(2, 2, 1)
(3, 2, 3)
(4, 0, 2)
(5, 0, 3)
(6, 1, 0)
(7, 1, 2)
(8, 3, 0)
Cargo Goal Positions (cargo_goal_pos):
(0, 0, 0)
(1, 2, 3)
(2, 3, 3)
(3, 1, 3)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
Alpha: 0.001
ID: 2

--------------------------------------------------



In [2]:
#最新 kiva 

from gurobipy import *
import csv
import time
from datetime import datetime

# 轉換為節點編號
def coord_to_node(i, j, grid_size):
    if i is None or j is None:
        return None
    return i * grid_size + j
    
def optimize_robot_paths(data):
    # Extract data from input
    T = data['T']
    grid_size = data['grid_size']
    num_nodes=grid_size * grid_size
    num_robots = data['num_robots']
    num_cargos = data['num_cargos']
    initial_positions = [(r, coord_to_node(i, j, grid_size)) for r, i, j in data['initial_positions']]
    cargo_init_pos = [(c, coord_to_node(i, j, grid_size)) for c, i, j in data['cargo_init_pos']]
    cargo_goal_pos = [(c, coord_to_node(i, j, grid_size)) for c, i, j in data['cargo_goal_pos']]
    alpha = data['alpha']
    scenario_id =  data['id']
    has_goal = {c: goal_node is not None for c, goal_node in cargo_goal_pos}

   

   
    m = Model("Robot and Cargo Path Planning")

   
    # 創建節點之間的連接關係，只允許上下左右移動
    edges = []
    for i in range(num_nodes):
        row, col = i // grid_size, i % grid_size
        if col < grid_size - 1:  # 水平連接
            edges.append((i, i+1))
        if row < grid_size - 1:  # 垂直連接
            edges.append((i, i+grid_size))
    
    # 轉向限制
    # 定義方向
    DIRECTIONS = {
        'right': (0, 1),
        'left': (0, -1),
        'down': (1, 0),
        'up': (-1, 0)
    }
    # 禁止連續移動的方向對應表
    INVALID_PAIRS = {
        'right': ['up', 'down'],   # 向右後不能向上或向下
        'left': ['up', 'down'],    # 向左後不能向上或向下
        'up': ['left', 'right'],   # 向上後不能向左或向右
        'down': ['left', 'right']  # 向下後不能向左或向右
    }
    # 獲取移動方向
    def get_direction(edge):
        i, j = edge
        i_row, i_col = i // grid_size, i % grid_size
        j_row, j_col = j // grid_size, j % grid_size
        return (j_row - i_row, j_col - i_col)
    
    # 方向變量
    dir_vars = m.addVars(num_robots, len(DIRECTIONS), T, vtype=GRB.BINARY, name="direction")
    # 建立雙向邊
    bidirectional_edges = edges + [(j, i) for i, j in edges]
    
    x = m.addVars(num_robots, num_nodes, num_nodes, T, vtype=GRB.BINARY, name="robot_move")
    pos = m.addVars(num_robots, num_nodes, T+1, vtype=GRB.BINARY, name="robot_pos")
    c_pos = m.addVars(num_cargos, num_nodes, T+1, vtype=GRB.BINARY, name="cargo_pos")
    z = m.addVars(num_robots, num_cargos, T+1, vtype=GRB.BINARY, name="carrying")
    y = m.addVars(num_cargos, T+1, vtype=GRB.BINARY, name="finish")
    max_arrival_time = m.addVar(vtype=GRB.INTEGER, name="max_arrival_time")
    
    # 機器人總移動次數權重
    alpha = 0.001
    
    
    # Objective function
    m.setObjective(
        max_arrival_time + 
        alpha * quicksum(x[r, i, j, t] for r in range(num_robots) 
                         for i, j in bidirectional_edges for t in range(T)),
        GRB.MINIMIZE
    )


    # Constraints

    for t in range(T):
        for r in range(num_robots):
            # 確保每個時間步只能有一個方向
            m.addConstr(quicksum(dir_vars[r, d, t] for d in range(len(DIRECTIONS))) <= 1)
            m.addConstr(quicksum(x[r, i, j, t] for i, j in bidirectional_edges) <= 1) #每個時間步動一次
    
            # 將邊的使用與方向變量關聯
            for i, j in bidirectional_edges:
                edge_dir = get_direction((i, j))
                for d, dir_vector in enumerate(DIRECTIONS.values()):
                    if edge_dir == dir_vector:
                        m.addConstr(x[r, i, j, t] <= dir_vars[r, d, t])
                        
            for n in range(num_nodes):
                # 如果機器人在時間t移動，必須使用合法的邊
                m.addConstr(
                    quicksum(x[r, i, j, t] for i, j in bidirectional_edges if i == n) <= pos[r, n, t]
                )
                m.addConstr(# 確保機器人位置轉換只發生在合法的邊上
                    pos[r, n, t+1] <= pos[r, n, t] + 
                    quicksum(x[r, i, n, t] for i, j in bidirectional_edges if j == n)
                )


    # 機器人初始位置
    for r, node in initial_positions:
        m.addConstr(pos[r, node, 0] == 1)
    
    # 貨物初始位置
    for c, node in cargo_init_pos:
        m.addConstr(c_pos[c, node, 0] == 1)
    
    # 貨物目標位置和到達時間約束
    for c, goal_node in cargo_goal_pos:
        if has_goal[c]:  # 只為有目標位置的貨物添加約束
            m.addConstr(y[c, T] == 1)  # 確保在最後時刻T時，有目標的貨物都到達終點
            for t in range(1, T+1):
                m.addConstr(max_arrival_time >= t * (y[c, t] - y[c, t-1]))
                m.addConstr(y[c, t] <= c_pos[c, goal_node, t])
                m.addConstr(y[c, t] >= y[c, t-1])
       
    
    # 機器人移動流量平衡
    for t in range(T):
        for r in range(num_robots):
            for n in range(num_nodes):
                m.addConstr(pos[r, n, t] + 
                            quicksum(x[r, i, n, t] for i, j in bidirectional_edges if j == n) ==
                            pos[r, n, t+1] + 
                            quicksum(x[r, n, j, t] for i, j in bidirectional_edges if i == n))
    
    
    # 每個節點在每個時刻只能有一個貨物、一個機器人
    for t in range(T+1):
        for n in range(num_nodes):
            m.addConstr(quicksum(pos[r, n, t] for r in range(num_robots)) <= 1)
            m.addConstr(quicksum(c_pos[c, n, t] for c in range(num_cargos)) <= 1)
    
    
    # 機器人和貨物在每個時刻只能在一個節點
    for t in range(T+1):
        for r in range(num_robots):
            m.addConstr(quicksum(pos[r, n, t] for n in range(num_nodes)) == 1)
        for c in range(num_cargos):
            m.addConstr(quicksum(c_pos[c, n, t] for n in range(num_nodes)) == 1)
    
    # 貨物移動約束
    for t in range(T):
        for c in range(num_cargos):
            for n in range(num_nodes):
                m.addConstr(c_pos[c, n, t+1] <= c_pos[c, n, t] + 
                            quicksum(z[r, c, t] * (pos[r, n, t+1]) 
                                    for r in range(num_robots)))
    
    # 機器人搬運貨物約束
    for t in range(T+1):
        for r in range(num_robots):
            for c in range(num_cargos):
                m.addConstr(z[r, c, t] <= quicksum(pos[r, n, t] * c_pos[c, n, t] 
                                                  for n in range(num_nodes)))
    
    # 搬運數量限制
    for t in range(T+1):
        for r in range(num_robots):
            m.addConstr(quicksum(z[r, c, t] for c in range(num_cargos)) <= 1)
        for c in range(num_cargos):
            m.addConstr(quicksum(z[r, c, t] for r in range(num_robots)) <= 1)
    
    # 避免機器人互換位置
    for t in range(T):
        for i, j in bidirectional_edges:
            m.addConstr(quicksum(x[r1, i, j, t] for r1 in range(num_robots)) +
                        quicksum(x[r2, j, i, t] for r2 in range(num_robots)) <= 1)

        # 限制機器人連續方向的約束
    # for t in range(T-1):
    #     for r in range(num_robots):
    #         for d1 in DIRECTIONS:
    #             for d2 in INVALID_PAIRS[d1]:
    #                 # 若 t 時刻向 d1 移動，則 t+1 不能向 d2 移動
    #                 m.addConstr(
    #                     dir_vars[r, list(DIRECTIONS).index(d1), t] +
    #                     dir_vars[r, list(DIRECTIONS).index(d2), t+1] <= 1
    #                 )
                        
    # 機器人轉向限制
    def is_perpendicular(edge1, edge2):
        i1, j1 = edge1
        i2, j2 = edge2
        return abs((j1 - i1) - (j2 - i2)) == grid_size - 1
    
    # 節點同時移動衝突約束
    for t in range(T):
        for n in range(num_nodes):
            row, col = n // grid_size, n % grid_size
            
            # 獲取所有可能的水平和垂直移動
            horizontal_moves = []
            vertical_moves = []
            
            # 檢查水平移動（左右）
            if col > 0:  # 可以從左側移入/移出
                horizontal_moves.append((n-1, n))  # 左到右
                horizontal_moves.append((n, n-1))  # 右到左
            if col < grid_size - 1:  # 可以從右側移入/移出
                horizontal_moves.append((n+1, n))  # 右到左
                horizontal_moves.append((n, n+1))  # 左到右
                
            # 檢查垂直移動（上下）
            if row > 0:  # 可以從上方移入/移出
                vertical_moves.append((n-grid_size, n))  # 上到下
                vertical_moves.append((n, n-grid_size))  # 下到上
            if row < grid_size - 1:  # 可以從下方移入/移出
                vertical_moves.append((n+grid_size, n))  # 下到上
                vertical_moves.append((n, n+grid_size))  # 上到下
            
            # 添加約束：水平移入和垂直移出不能同時發生
            for h_move in horizontal_moves:
                if h_move[1] == n:  # 水平移入
                    for v_move in vertical_moves:
                        if v_move[0] == n:  # 垂直移出
                            for r1 in range(num_robots):
                                for r2 in range(num_robots):
                                    m.addConstr(
                                        x[r1, h_move[0], h_move[1], t] + 
                                        x[r2, v_move[0], v_move[1], t] <= 1
                                    )
            
            # 添加約束：垂直移入和水平移出不能同時發生
            for v_move in vertical_moves:
                if v_move[1] == n:  # 垂直移入
                    for h_move in horizontal_moves:
                        if h_move[0] == n:  # 水平移出
                            for r1 in range(num_robots):
                                for r2 in range(num_robots):
                                    m.addConstr(
                                        x[r1, v_move[0], v_move[1], t] + 
                                        x[r2, h_move[0], h_move[1], t] <= 1
                                    )

    # Optimize
    m.setParam('TimeLimit', 14400)
    m.setParam('MIPFocus', 2)
    m.optimize()

    solving_time=m.Runtime

    return m, x, pos,c_pos, z, y, scenario_id, solving_time



def save_metrics_to_csv(metrics):
    # Generate filename with current timestamp
    current_time = datetime.now()
    filename = current_time.strftime("%Y%m%d_%H%M%S") + "_optimization_metrics.csv"
    
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['Number of Robots', 'Number of Cargos', 'Number of Targets','Scenario ID',
                      'Objective Value','Gap', 'Solving Time (s)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for metric in metrics:
            writer.writerow(metric)


def print_results(m, x, pos,c_pos, z, y, data):
    if m.status == GRB.OPTIMAL:
        def node_to_coord(node):
            return (node // grid_size, node % grid_size)

        T = data['T']
        grid_size = data['grid_size']
        num_robots = data['num_robots']
        num_cargos = data['num_cargos']
        cargo_goal_pos = [(c, coord_to_node(i, j, grid_size)) for c, i, j in data['cargo_goal_pos']]
        has_goal = {c: goal_node is not None for c, goal_node in cargo_goal_pos}
        num_nodes=grid_size * grid_size

       
        for t in range(T+1):
            print(f"時間 {t}")
            
            # 網格顯示
            grid = [[[] for _ in range(grid_size)] for _ in range(grid_size)]
            
            # 機器人位置
            for r in range(num_robots):
                for n in range(num_nodes):
                    if pos[r, n, t].x > 0.5:
                        i, j = node_to_coord(n)
                        grid[i][j].append(f"R{r+1}")
            
            # 貨物位置
            for c in range(num_cargos):
                for n in range(num_nodes):
                    if c_pos[c, n, t].x > 0.5:
                        i, j = node_to_coord(n)
                        grid[i][j].append(f"C{c+1}")
            
            # 輸出網格
            for i, row in enumerate(grid):
                print("+", end="")
                for _ in range(grid_size):
                    print("--------+", end="")
                print()
                print("|", end="")
                for j, cell in enumerate(row):
                    content = ",".join(cell) if cell else " "
                    print(f" {content:6} |", end="")
                print()
            print("+", end="")
            for _ in range(grid_size):
                print("--------+", end="")
            print("\n")
            
            # 輸出詳細信息
            for r in range(num_robots):
                robot_nodes = [n for n in range(num_nodes) if pos[r, n, t].x > 0.5]
                if robot_nodes:
                    i, j = node_to_coord(robot_nodes[0])
                    print(f"機器人{r+1}在 ({i},{j})", end="")
                    
                    carrying = [c+1 for c in range(num_cargos) if z[r, c, t].x > 0.5]
                    if carrying:
                        print(f", 正在搬運貨物{carrying[0]}", end="")
                    print()
            
            # 貨物信息
            for c in range(num_cargos):
                cargo_nodes = [n for n in range(num_nodes) if c_pos[c, n, t].x > 0.5]
                if cargo_nodes:
                    i, j = node_to_coord(cargo_nodes[0])
                    status = ""
                    
                    # 檢查是否有目標位置
                    goal_node = cargo_goal_pos[c][1]
                    if goal_node is not None:
                        if cargo_nodes[0] == goal_node:
                            status = ", 已到達目標位置"
                        else:
                            goal_i, goal_j = node_to_coord(goal_node)
                            status = f", 目標位置為({goal_i},{goal_j})"
                    else:
                        status = ", 無目標位置"
                    
                    print(f"貨物{c+1}在 ({i},{j}){status}")
            print()
    else:
        print("無法找到解")


def main(all_data):
    metrics = []
    
    for data in all_data:
        print(f"Processing scenario with {data['num_robots']} robots and {data['num_cargos']} cargos")
        m, x, pos,c_pos, z, y, scenario_id, solving_time = optimize_robot_paths(data)
        
        
        metric = {
            'Number of Robots': data['num_robots'],
            'Number of Cargos': data['num_cargos'],
            'Number of Targets': data['num_targets'],
            'Scenario ID': scenario_id,
            'Objective Value': round(m.objVal, 3)  if m.SolCount > 0 else None, 
            'Gap':f"{round(m.MIPGap * 100, 1)}%" if m.SolCount > 0 else None,
            'Solving Time (s)': round(solving_time, 2)
        }
        metrics.append(metric)
        
        if m.status == GRB.OPTIMAL:
            print_results(m, x, pos,c_pos, z, y, data)
        print("\n" + "="*50 + "\n")
    
    # Save metrics to CSV
    save_metrics_to_csv(metrics)


if __name__ == "__main__":
    
    main(all_data)

Processing scenario with 6 robots and 9 cargos
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-10
Set parameter TimeLimit to value 14400
Set parameter MIPFocus to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 39705 rows, 22660 columns and 107359 nonzeros
Model fingerprint: 0x41c5cc61
Model has 2430 quadratic constraints
Variable types: 0 continuous, 22660 integer (22659 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 34325 rows and 16923 columns
Presolve time: 1.06s
Presolved: 39306 rows, 21694 columns, 1292